In [31]:
import time
import requests
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [32]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"

BASE_URL = "https://www.bcn.cl/historiapolitica/resenas_parlamentarias/index.html"

PARAMS_EX = {
    "categ": "por_periodo",
    "periodo": "1990-2018",
}

PARAMS_EJERCICIO = {
    "categ": "en_ejercicio",
    "filtros": "1",
}
HEADERS = {
    "User-Agent": "Mozilla/5.0",
}

In [33]:
def fetch_page(params: dict, pagina: int):
    params = {**params, "pagina": str(pagina)}
    r = requests.get(BASE_URL, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text, r.url


def parse_cards(html: str, base_url: str):
    soup = BeautifulSoup(html, "html.parser")
    items = []
    for li in soup.select("#contenedorResultados li"):
        a = li.find("a", href=True)
        if not a:
            continue
        url = urljoin(base_url, a["href"])
        nombre = a.get_text(strip=True)
        items.append({"nombre_en_lista": nombre, "url_wiki": url})
    return items


def scrape_bcn_list(params: dict, pagina_inicial=1, pagina_final=50, sleep=1.0):
    """
    Recorre todas las páginas para un conjunto de parámetros (periodo, categoría, etc.)
    """
    rows = []
    for p in range(pagina_inicial, pagina_final + 1):
        html, resolved_url = fetch_page(params, p)
        cards = parse_cards(html, resolved_url)
        if not cards:
            print(f"Fin del listado en página {p}")
            break
        for it in cards:
            it["pagina"] = p
        rows.extend(cards)
        print(f"Página {p}: {len(cards)} registros")
        time.sleep(sleep)
    return pd.DataFrame(rows)

In [34]:
df_ex = scrape_bcn_list(PARAMS_EX, pagina_inicial=1, pagina_final=20)
df_activos = scrape_bcn_list(PARAMS_EJERCICIO, pagina_inicial=1, pagina_final=10)
df_total = pd.concat([df_ex, df_activos], ignore_index=True).drop_duplicates(subset=["nombre_en_lista"], keep="first")

Página 1: 44 registros
Página 2: 44 registros
Página 3: 44 registros
Página 4: 44 registros
Página 5: 44 registros
Página 6: 44 registros
Página 7: 44 registros
Página 8: 44 registros
Página 9: 44 registros
Página 10: 44 registros
Página 11: 44 registros
Página 12: 44 registros
Página 13: 44 registros
Página 14: 44 registros
Página 15: 7 registros
Fin del listado en página 16
Página 1: 44 registros
Página 2: 44 registros
Página 3: 44 registros
Página 4: 44 registros
Página 5: 29 registros
Fin del listado en página 6


In [35]:
df_total.to_csv(DATA_DIR / "parlamentarios.csv", index=False, encoding="utf-8-sig")